In [1]:
from functions_trained_reservoir import *

from reservoirpy.nodes import Reservoir,FORCE,Ridge
from reservoirpy.mat_gen import Initializer,_scale_spectral_radius
from reservoirpy.observables import spectral_radius
from tqdm import tqdm

from ReccurentNetwork import *

import numpy as np
from numpy.linalg import eig

import spicy as sp
import matplotlib.pyplot as plt

# Réseaux de neurones

On cherche ici dans un premier temps à reprdoduire les figures les plus simples de l'article. 

Les mesures de l'article sont effectuées sur 2 réservoirs : 

  - le premier est dit entrainé : on va alors entrainer le réservoir et la sortie
  - le second lui est dit non entrainé : on entraine seulement la sortie
    

Ce premier script s'intéresse donc à la reproduction de l'entrainement des figures 3, 4 et 5 ainsi qu'aux résultas obtenus sur un réservoir entrainé.
 
La librairie utilisé pour entrainer le réservoir est prise dans autre article ReScience : https://github.com/ReScience-Archives/Vitay-2016

Copyright (c) 2016 Julien Vitay julien.vitay@informatik.tu-chemnitz.de

## Expériences

Deux expériences quasiment similaires sont proposés pour un réservoir totalement entrainé. La seule différence entre ces dernières sont le nombre de neurones utilisés dans le réservoir : 

   - Soit N = 2100 pour des résultats robustes entre toutes les données pour un sujet
   - Soit N = 4000 pour des résultats robustes entre tous les sujets (5 au maximum)
    
Nous traiterons le premier  cas dans ce papier.

##### Méthode d'entrainement

Pour ce qui est de l'entrainement du réservoir nous allons procéder en 2 étapes. En effet, nous séparons notre entrée en deux parties : l'époque sensorielle et l'époque motricielle. Nous séparons l'entrainement de ces 2 époques que je vais détailler ci dessous.

###### Epoque sensorielle : 

Pour un sujet donné et parmit toutes les entrées différentes, pour chaque chiffre on selectionne une entrée moyenne. Cette entrée est injectée dans le réservoir sans bruit et sans target, on obtient alors une trajectoire resultante de dimension (??) pour chaque chiffre. Ces 10 trajectoires sont appelées les "innate trajectories" et seront les targets pour toutes les autres entrées durant l'entrainement du l'époque sensorielle en faisant bien attention à  ce que la dimension d'entrée des nouvelles entrées soit cohérente avec celle de l'innate trajectorie, on procédera alors pas une déformation linéaire de l'innate trajectory. De plus, du bruit sera présent durant l'entrainement.

###### Epoque motrice

target = une target par chiffre peu importe l'entrée ou le sujet
c'est quoi la target ?? 
comprends la transition de 300 ms aussi


###### Test
Les test effectués dans ce script seront fait sur le sujet 1

Données générales 

In [2]:
data = extract_data() #Données d'entrée
transcription = extract_target() #Données d'arrivée 

#Les données suivantes sont utilisés dans les fonctions mais pas dans le script ! 

freq_ent = 6*10**3  #fréquence d'entrée
T_ent = 1/freq_ent  #période d'entrée

freq_out = 10**3 #fréquence de sortie 
T_out = 1/freq_out #période de sortie

ampl_int = 5 #Amplitude d'entrée

Données du réservoir

In [3]:
N = 2100 #Nombre de neurones dans le réservoir
M = 12 #Nombres d'entrées
No = 3 #Nombres de sorties
tau = 25*10e-3
dt = 1e-3
lr = dt/tau
sr = 1.6 #spectral radius --> gain ?
g = 1.6 #le gain du réseau ?                             #je comprends pas trop cette donnée. 
pc = 0.2 # connectivité entre les neurones (pc dans l'article)
SD = g/np.sqrt(pc*N)
mu = 0
I0 = 0.05 #Bruit dans le réservoir

Wo = normal(No,N,loc=mu,scale = 1/np.sqrt(N))    #Matrice de sortie : Non utilisée


Création de la matrice du réservoir : avec pas d'autapse
Puis bricolage pour réxupérer le rayon spectral voulu

In [4]:
W_r.init = Initializer(W_r)

W = _scale_spectral_radius(W_r.init, [N,N], sr)
W = W.toarray()

#Vérification du rayon spectral

#a,b=eig(W)
#print(np.max(np.abs(a)))


Création de la matrice d'entrée : chaque entrée k est projetée sur le neurone $(k-1)\frac{N}{M} + 1$ jusqu'au neurone $k\frac{N}{M}$ selon une loi normale 

In [5]:
Win = W_in(N,M)

### Epoque sensorielle 

Récupération des cocleograms avec une taille moyenne pour chaque chiffre 

In [6]:
median_entrance_cocleogram = median_cocleogram(data,sujet=[1])

Création du réservoir

In [7]:
network = RecurrentNetwork(
    W_in = Win,
    W_rec = W,
    W_out = Wo,
    Ni = M, # Number of inputs
    N = N, # Number of recurrent neurons
    No = No, # Number of read-out neurons
    tau = 25, # Time constant of the neurons
    g = g, # Synaptic strength scaling
    pc = pc, # Connection probability
    Io = I0, # Noise variance
    delta = 1.0, # Initial diagonal value of the P matrix                                 # ?
    P_plastic = 0.6, # Percentage of neurons receiving plastic synapses                   # ?
)

Détermination des trajectoires inées  

In [8]:
innate_trajectories_sensor = []
innate_trajectories_motor = []
for i in range(len(median_entrance_cocleogram)):
    
    #Récupération du cocleogram moyen
    impulse = np.expand_dims(median_entrance_cocleogram[i], axis=2)
    
    #Récupération des trajectoires innées de l'époque sensorielle et de l'époque motricielle ???? 
    initial_trajectory, initial_output = network.simulate(stimulus=impulse, noise=False)
    
    #Ensuite on les trie
    innate_trajectories_sensor.append(np.squeeze(initial_trajectory,axis = 2))
    innate_trajectories_motor.append(np.squeeze(initial_output,axis = 2))

100%|███████████████████████████████████████████████████████████████████████████████| 712/712 [00:00<00:00, 913.82it/s]


#### Entrainement de l'epoque sensorielle.

Récupération des données d'entrées et des targets, ici on entraîne sur un sujet, 3 entrées et les 10 chiffres

In [9]:
X_input = []
X_target = []
for indS in [1] : 
    for indU in range(1,4):
        for indD in range(10):
            X_input.append(np.expand_dims(np.transpose(cocleogram(indS, indU, indD,data)), axis=2))
            X_target.append(np.expand_dims(linear_warping(indS,indU,indD,data,innate_trajectories_sensor[indD]),axis=2))

step = len(X_target)  #Nombre d'entrainement à effectuer

Entrainement

In [10]:
for i in range(step): 
    
    end = np.shape(X_input[i])[0]
    
    print(end)
    print(i)
    
    network.simulate(stimulus=X_input[i],
                 noise = True,
                 trajectory=X_target[i],
                 learn_start=0,
                 learn_stop=end)


531
0


  3%|██▍                                                                              | 16/531 [00:07<04:12,  2.04it/s]


KeyboardInterrupt: 

Sauvegarde des données

In [ ]:
network.save('network_fig345.npz')

In [ ]:
from time import sleep
from tqdm import tqdm
for i in tqdm(range(10)):
    sleep(0.2)

In [17]:
reservoir = Reservoir(units = N,lr = lr,sr=sr,W=W_r.init,Win=Win,equation="external",input_bias=False,noise_rc = I0)

In [85]:
N = 4

W_r.init = Initializer(W_r)

W = _scale_spectral_radius(W_r.init, [N,N], 1.0)
W = W.toarray()

Wo = normal(N,1,loc=mu,scale = 1/np.sqrt(N))

Win = W_in(N,1)

x = np.random.uniform(-1.0, 1.0, (N, 1))

u = np.random.uniform(-1.0, 1.0, (N, 1))
y_target = np.random.uniform(-1.0, 1.0, (N, 1))

e = y_target - y
r = np.tanh(x)

P = np.identity(N)


In [86]:
def _rls(P, x, e):
    """Recursive Least Squares learning rule."""
    k = np.dot(P, x)
    xPx = np.dot(x.T, k)
    c = float(1.0 / (1.0 + xPx))
    P = P - c * np.outer(k, k)

    dw = -c * np.outer(e, k)

    return dw, P

def x_kplus1(Win,u,W,x):
    return np.tanh(np.dot(Win,u) + np.dot(W,x))

def y_k(Wo,x_kplus1):
    return np.dot(Wo,x_kplus1)

def error(y,y_target):
    return y - y_target


x_1 = x_kplus1(Win,u,W,x)

y_1 = y_k(Wo,x_1)
e = error(y_1,y_target)

print(e)

dw,P = _rls(P,x_1,e)

W -= dw

x_2 = x_kplus1(Win,u,W,x_1)
y_2 = y_k(Wo,x_2)
e = error(y_2,y_target)

print(e)

dw,P = _rls(P,x_2,e)

W -= dw



[[-0.33201136]
 [-0.86242004]
 [ 1.82359531]
 [ 1.61344042]]
[[-1.50450082]
 [-0.37381499]
 [-0.60264251]
 [-1.80833834]]


In [63]:


dw,P = _rls(P,r,e)

[[ 0.72306831 -0.28072445  0.05308362 -0.28217433]
 [-0.28072445  0.71543085  0.05381064 -0.2860389 ]
 [ 0.05308362  0.05381064  0.98982467  0.05408856]
 [-0.28217433 -0.2860389   0.05408856  0.71248377]]
[[ 0.7150435  -0.28885916  0.05462186 -0.29035106]
 [-0.28885916  0.70718472  0.05536994 -0.29432761]
 [ 0.05462186  0.05536994  0.98952981  0.05565592]
 [-0.29035106 -0.29432761  0.05565592  0.70415225]]


In [64]:
print(e)

[[-0.3751006 ]
 [ 1.65180147]
 [ 1.13648591]
 [ 0.40238461]]


In [87]:
W

array([[-2.00109950e-01, -2.00109950e-01, -2.00735844e+05,
        -8.12440969e+04],
       [-1.29491738e+06,  1.53429642e-01,  5.99515621e-02,
         2.21001985e-01],
       [-5.89793556e-01, -5.89793556e-01, -3.49046635e-01,
        -2.67775571e-01],
       [-7.64724939e-01, -7.64724939e-01, -5.12283089e-01,
        -5.42731779e-02]])

In [88]:
x

array([[ 0.85933389],
       [ 0.71632935],
       [ 0.49512721],
       [-0.77347201]])

In [90]:
x = np.dot(W,x)